# 📚 The Great Gatsby - Vector Database Project

This notebook demonstrates how to convert F. Scott Fitzgerald's "The Great Gatsby" into a searchable vector database using ChromaDB and perform semantic search queries.

- Loading and preprocessing text data
- Creating embeddings with SentenceTransformers
- Storing vectors in ChromaDB
- Performing semantic search queries

## 1. Install and Import Required Libraries

In [1]:
import chromadb
from chromadb.utils import embedding_functions
from pathlib import Path
import requests
import re
from pprint import pprint

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 2. Download The Great Gatsby Text

We'll download the text from Project Gutenberg (public domain).

In [2]:
# Download The Great Gatsby from Project Gutenberg
url = "https://www.gutenberg.org/files/64317/64317-0.txt"
data_path = Path("data/great_gatsby.txt")

# Create data directory if it doesn't exist
data_path.parent.mkdir(exist_ok=True)

# Download the file if it doesn't exist
if not data_path.exists():
    print("📥 Downloading The Great Gatsby...")
    response = requests.get(url)
    response.raise_for_status()
    
    with open(data_path, 'w', encoding='utf-8') as f:
        f.write(response.text)
    print("✅ Downloaded successfully!")
else:
    print("✅ File already exists!")

# Read the file
with open(data_path, 'r', encoding='utf-8') as f:
    raw_text = f.read()

print(f"📊 Total characters: {len(raw_text)}")
print(f"📊 First 500 characters:\n{raw_text[:500]}")

📥 Downloading The Great Gatsby...
✅ Downloaded successfully!
📊 Total characters: 270822
📊 First 500 characters:
*** START OF THE PROJECT GUTENBERG EBOOK 64317 ***




                           The Great Gatsby
                                  by
                          F. Scott Fitzgerald


                           Table of Contents

I
II
III
IV
V
VI
VII
VIII
IX


                              Once again
                                  to
                                 Zelda


  Then wear the gold hat, if that will move her;
  If you can bounce high, bounce for her too,
  Till she cry “Lover, go


## 3. Clean and Preprocess the Text

Remove Project Gutenberg headers/footers and clean the text.

In [3]:
# Clean the text - remove Project Gutenberg header and footer
def clean_text(text):
    # Find the start of the actual book (after the Gutenberg header)
    start_markers = ["CHAPTER I", "Chapter I", "CHAPTER 1"]
    start_idx = 0
    for marker in start_markers:
        idx = text.find(marker)
        if idx != -1:
            start_idx = idx
            break
    
    # Find the end (before Gutenberg footer)
    end_markers = ["End of Project Gutenberg", "*** END OF THE PROJECT GUTENBERG"]
    end_idx = len(text)
    for marker in end_markers:
        idx = text.find(marker)
        if idx != -1:
            end_idx = idx
            break
    
    # Extract the main text
    clean = text[start_idx:end_idx]
    
    # Clean up extra whitespace
    clean = re.sub(r'\n{3,}', '\n\n', clean)
    clean = re.sub(r' {2,}', ' ', clean)
    
    return clean.strip()

cleaned_text = clean_text(raw_text)
print(f"📊 Cleaned text length: {len(cleaned_text)} characters")
print(f"📊 First 500 characters:\n{cleaned_text[:500]}")

📊 Cleaned text length: 269986 characters
📊 First 500 characters:
*** START OF THE PROJECT GUTENBERG EBOOK 64317 ***

 The Great Gatsby
 by
 F. Scott Fitzgerald

 Table of Contents

I
II
III
IV
V
VI
VII
VIII
IX

 Once again
 to
 Zelda

 Then wear the gold hat, if that will move her;
 If you can bounce high, bounce for her too,
 Till she cry “Lover, gold-hatted, high-bouncing lover,
 I must have you!”

 Thomas Parke d’Invilliers

 I

In my younger and more vulnerable years my father gave me some advice
that I’ve been turning over in my mind ever since.

“Whenev


## 4. Split Text into Chunks

We'll split the text into paragraphs for better semantic search results.

In [4]:
# Split into paragraphs (chunks)
def split_into_chunks(text, min_length=100):
    """Split text into paragraphs, filtering out very short ones."""
    # Split by double newlines (paragraphs)
    paragraphs = text.split('\n\n')
    
    # Filter out very short paragraphs
    chunks = [p.strip() for p in paragraphs if len(p.strip()) >= min_length]
    
    return chunks

chunks = split_into_chunks(cleaned_text)
print(f"📊 Number of chunks: {len(chunks)}")
print(f"📊 Average chunk length: {sum(len(c) for c in chunks) // len(chunks)} characters")
print(f"\n📝 Sample chunk (first one):\n{chunks[0][:300]}...")
print(f"\n📝 Sample chunk (middle one):\n{chunks[len(chunks)//2][:300]}...")

📊 Number of chunks: 769
📊 Average chunk length: 289 characters

📝 Sample chunk (first one):
Then wear the gold hat, if that will move her;
 If you can bounce high, bounce for her too,
 Till she cry “Lover, gold-hatted, high-bouncing lover,
 I must have you!”...

📝 Sample chunk (middle one):
“They’re such beautiful shirts,” she sobbed, her voice muffled in the
thick folds. “It makes me sad because I’ve never seen such—such
beautiful shirts before.”...


## 5. Initialize ChromaDB Client

Create a persistent client to store our vector database locally.

In [6]:
# Initialize ChromaDB with persistent storage
chroma_client = chromadb.PersistentClient(path="./vector_db")
print("✅ ChromaDB client initialized!")

✅ ChromaDB client initialized!


## 6. Create Sentence Transformer Embedding Function

Initialize the embedding model using SentenceTransformers.

In [7]:
# Create embedding function using SentenceTransformer
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

print("✅ Sentence Transformer embedding function created!")
print("📊 This model creates 384-dimensional embeddings")


✅ Sentence Transformer embedding function created!
📊 This model creates 384-dimensional embeddings


## 7. Create Vector Embeddings

Generate embeddings for all text chunks. This may take a minute...

In [8]:
# Generate embeddings for all chunks
print(f"🔄 Generating embeddings for {len(chunks)} chunks...")
vectors = sentence_transformer_ef(chunks)

print(f"✅ Generated {len(vectors)} embeddings!")
print(f"📊 Each embedding has {len(vectors[0])} dimensions")

🔄 Generating embeddings for 769 chunks...
✅ Generated 769 embeddings!
📊 Each embedding has 384 dimensions


## 8. Store Embeddings in ChromaDB Collection

Create a collection and add all chunks with their embeddings.

In [9]:
# Create unique IDs for each chunk
ids = [f"chunk_{i}" for i in range(len(chunks))]

# Uncomment to delete existing collection if needed
# chroma_client.delete_collection(name="great_gatsby")

# Create or get collection
collection = chroma_client.get_or_create_collection(name="great_gatsby")

print(f"📦 Adding {len(chunks)} chunks to ChromaDB...")

# Add documents to collection
collection.add(
    documents=chunks,
    ids=ids,
    embeddings=vectors,
)

print(f"✅ Successfully added documents to collection!")
print(f"📊 Collection count: {collection.count()}")

📦 Adding 769 chunks to ChromaDB...
✅ Successfully added documents to collection!
📊 Collection count: 769


## 9. Query the Vector Database

Now let's search for relevant passages using semantic search!

In [10]:
# Query 1: Find passages about Gatsby's parties
query = "Gatsby's extravagant parties with music and dancing"
query_embedding = sentence_transformer_ef([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=5, # how many results to return
)

print("🔍 Query:", query)
print("\n" + "="*80)
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n📄 Result {i+1} (Similarity: {1-distance:.4f})")
    print("-" * 80)
    print(doc[:300] + "..." if len(doc) > 300 else doc)
    print()

🔍 Query: Gatsby's extravagant parties with music and dancing


📄 Result 1 (Similarity: 0.3714)
--------------------------------------------------------------------------------
I believe that on the first night I went to Gatsby’s house I was one
of the few guests who had actually been invited. People were not
invited—they went there. They got into automobiles which bore them out
to Long Island, and somehow they ended up at Gatsby’s door. Once there
they were introduced by ...


📄 Result 2 (Similarity: 0.2974)
--------------------------------------------------------------------------------
Daisy and Gatsby danced. I remember being surprised by his graceful,
conservative foxtrot—I had never seen him dance before. Then they
sauntered over to my house and sat on the steps for half an hour,
while at her request I remained watchfully in the garden. “In case
there’s a fire or a flood,” she ...


📄 Result 3 (Similarity: 0.1896)
-------------------------------------------------------------------

In [11]:
# Query 2: Find passages about the green light
query = "the green light at the end of the dock"
query_embedding = sentence_transformer_ef([query])

results = collection.query(
    query_embeddings=query_embedding,
    n_results=3,
)

print("🔍 Query:", query)
print("\n" + "="*80)
for i, (doc, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"\n📄 Result {i+1} (Similarity: {1-distance:.4f})")
    print("-" * 80)
    print(doc[:400] + "..." if len(doc) > 400 else doc)
    print()

🔍 Query: the green light at the end of the dock


📄 Result 1 (Similarity: 0.2204)
--------------------------------------------------------------------------------
“If it wasn’t for the mist we could see your home across the bay,”
said Gatsby. “You always have a green light that burns all night at
the end of your dock.”


📄 Result 2 (Similarity: 0.0462)
--------------------------------------------------------------------------------
With an effort Wilson left the shade and support of the doorway and,
breathing hard, unscrewed the cap of the tank. In the sunlight his
face was green.


📄 Result 3 (Similarity: -0.2387)
--------------------------------------------------------------------------------
Daisy put her arm through his abruptly, but he seemed absorbed in what
he had just said. Possibly it had occurred to him that the colossal
significance of that light had now vanished forever. Compared to the
great distance that had separated him from Daisy it had seemed very
near to her, almost 

## 11. Alternative Query Method


In [12]:
# Alternative method - let ChromaDB handle the embedding
results = collection.query(
    query_texts=["Gatsby's mysterious wealth and background"],
    n_results=3,
)

pprint(results)

{'data': None,
 'distances': [[0.6270289421081543, 0.6323080062866211, 0.6470474004745483]],
 'documents': [['Something in her tone reminded me of the other girl’s “I '
                'think he\n'
                'killed a man,” and had the effect of stimulating my '
                'curiosity. I would\n'
                'have accepted without question the information that Gatsby '
                'sprang from\n'
                'the swamps of Louisiana or from the lower East Side of New '
                'York. That\n'
                'was comprehensible. But young men didn’t—at least in my '
                'provincial\n'
                'inexperience I believed they didn’t—drift coolly out of '
                'nowhere and\n'
                'buy a palace on Long Island Sound.',
                'There was a small picture of Gatsby, also in yachting '
                'costume, on the\n'
                'bureau—Gatsby with his head thrown back defiantly—taken '
                'appar

## 12. Database Statistics

Let's check the statistics of our vector database.

In [13]:
print("📊 Vector Database Statistics")
print("="*80)
print(f"Collection Name: {collection.name}")
print(f"Total Documents: {collection.count()}")
print(f"Embedding Dimensions: 384")
print(f"Embedding Model: all-MiniLM-L6-v2")
print(f"Storage Path: ./gatsby_vector_db")
print("\n✅ The Great Gatsby is now fully searchable as a vector database!")

📊 Vector Database Statistics
Collection Name: great_gatsby
Total Documents: 769
Embedding Dimensions: 384
Embedding Model: all-MiniLM-L6-v2
Storage Path: ./gatsby_vector_db

✅ The Great Gatsby is now fully searchable as a vector database!
